In [7]:
import cv2
from Model import imageProcess as imp
from Model import ocrProcess as op

import cv2
import torch
from paddleocr import PaddleOCR
import numpy as np
import easyocr
import pytesseract

easy_ocr = easyocr.Reader(['en'])  # 'en' untuk bahasa Inggris

[2025-07-13 21:29:26,586] [ WARNING] easyocr.py:80 - Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [9]:
image = cv2.imread("Plate/Plat159_jpg.rf.374e29e9dfef0d668b90d5dd9e28affc.jpg")

plat_number, plat_conf = op.f_ocr_process(image)
# plat_number, plat_conf = op.f_easyocr_process(image)

print(f"plate number : {plat_number}")

[2025/07/13 21:31:29] ppocr DEBUG: dt_boxes num : 3, elapsed : 0.06661677360534668
[2025/07/13 21:31:29] ppocr DEBUG: cls num  : 3, elapsed : 0.007999897003173828
[2025/07/13 21:31:29] ppocr DEBUG: rec_res num  : 3, elapsed : 0.07012343406677246
plate number : B3284EFX


In [8]:
image = cv2.imread("Plate/Plat159_jpg.rf.374e29e9dfef0d668b90d5dd9e28affc.jpg")

plat_detect = ""
plat_predictval = ""

# 1. Resize (jika terlalu kecil)
if image.shape[0] < 50 or image.shape[1] < 150:
    image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

# 2. Grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 3. Bilateral Filter → mengurangi noise, tapi menjaga edge
gray = cv2.bilateralFilter(gray, 11, 17, 17)

# 4. Adaptive Thresholding → lebih fleksibel dari OTSU
thresh = cv2.adaptiveThreshold(
    gray, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV,
    25, 15
)

# 5. Morphological Operation (Optional)
kernel = np.ones((2, 2), np.uint8)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# 6. EasyOCR Reading
results = easy_ocr.readtext(thresh)
print(f"result : {results}")
# if not results:
#     print("Gagal dapat hasil OCR")
# else:
#     # Ambil hasil dengan confidence tertinggi
#     best_result = max(results, key=lambda r: r[2])
#     _, text, confidence = best_result

#     # plat_detect = f_remove_special_char(text)
#     plat_predictval = confidence

result : [([[np.int32(133), np.int32(175)], [np.int32(638), np.int32(175)], [np.int32(638), np.int32(334)], [np.int32(133), np.int32(334)]], '928/ EF8', np.float64(0.7716032061128456)), ([[np.int32(353), np.int32(370)], [np.int32(463), np.int32(370)], [np.int32(463), np.int32(449)], [np.int32(353), np.int32(449)]], '03', np.float64(0.9784326759738196)), ([[np.int32(511), np.int32(367)], [np.int32(623), np.int32(367)], [np.int32(623), np.int32(459)], [np.int32(511), np.int32(459)]], '20', np.float64(0.9738883736950135))]
